In [1]:
# check reaction family

In [2]:
import os
import re
import copy
import glob
import itertools

from rmgpy.exceptions import ActionError
import rmgpy.reaction
import rmgpy.chemkin
import rmgpy.data.kinetics

import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

In [3]:
# load the database
# load the thermo database,
# this is minimal for better speed
thermo_libs = [
    'primaryThermoLibrary',
]

thermo_library_path = os.path.join(rmgpy.settings['database.directory'], 'thermo')
thermo_database = rmgpy.data.thermo.ThermoDatabase()
thermo_database.load(
    thermo_library_path,
    libraries=thermo_libs
)


# load the families
ref_library_path = os.path.join(rmgpy.settings['database.directory'], 'kinetics')
kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
kinetics_database.load(
    ref_library_path,
    libraries=[],
    families='all'
)

# load the entire database
ref_db = rmgpy.data.rmg.RMGDatabase()
ref_db.kinetics = kinetics_database
ref_db.thermo = thermo_database

In [11]:
# load minimal ethane example
chemkin_file = '/home/moon/rmg/my_examples/ethane/chemkin/chem_annotated.inp'
dict_file = '/home/moon/rmg/my_examples/ethane/chemkin/species_dictionary.txt'
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_file, dict_file, use_chemkin_names=True)

In [70]:
def relabel(input_r, family):
    input_reaction = copy.deepcopy(input_r)
    
    
    # copied from AutoTST.autotst.reaction.py
    def get_rmg_mol(smile):
        smiles_conversions = {
                "[CH]": "[CH...]",
                "CARBONMONOXIDE": "[C-]#[O+]"
            }

        if smile.upper() in list(smiles_conversions.keys()):
            smile = smiles_conversions[smile.upper()]
        return rmgpy.molecule.Molecule(smiles=smile).generate_resonance_structures()
    
    rmg_reactants = [get_rmg_mol(sp.smiles) for sp in input_reaction.reactants]
    rmg_products = [get_rmg_mol(sp.smiles) for sp in input_reaction.products]

    combos_to_try = list(itertools.product(
                list(itertools.product(*rmg_reactants)),
                list(itertools.product(*rmg_products))
            ))
    
    for rmg_reactants, rmg_products in combos_to_try:

        test_reaction = rmgpy.reaction.Reaction(
            reactants=list(rmg_reactants),
            products=list(rmg_products)
        )

        try:
            labeled_r, labeled_p = ref_db.kinetics.families[family].get_labeled_reactants_and_products(
                test_reaction.reactants,
                test_reaction.products
            )
            
            if labeled_r is None or labeled_p is None:
                continue
            
            if input_reaction.reactants[0].molecule[0].is_isomorphic(labeled_r[0]):
                input_reaction.reactants[0].molecule[0] = labeled_r[0]
                input_reaction.reactants[1].molecule[0] = labeled_r[1]
            else:
                input_reaction.reactants[0].molecule[0] = labeled_r[1]
                input_reaction.reactants[1].molecule[0] = labeled_r[0]
                
                
            if input_reaction.products[0].molecule[0].is_isomorphic(labeled_p[0]):
                input_reaction.products[0].molecule[0] = labeled_p[0]
                input_reaction.products[1].molecule[0] = labeled_p[1]
            else:
                input_reaction.products[0].molecule[0] = labeled_p[1]
                input_reaction.products[1].molecule[0] = labeled_p[0]
                
                
#             if not valid_labels(input_reaction):
#                 print(f'Bad labeling on {input_reaction}')
            return input_reaction


        except ActionError:
            pass
    return False

In [14]:
# print the details of one reaction to copy for later
index = 1
print(reaction_list[index].family)
print(reaction_list[index])

for species in reaction_list[index].reactants + reaction_list[index].products:
    print(species.smiles)

H_Abstraction
CH3(4) + ethane(1) <=> C(3) + C2H5(5)
[CH3]
CC
C
C[CH2]


In [64]:
# construct a reaction by hand (or load from the NIST model)
reactant1 = rmgpy.species.Species(smiles='[CH3]')
reactant2 = rmgpy.species.Species(smiles='CC')
product1 = rmgpy.species.Species(smiles='C')
product2 = rmgpy.species.Species(smiles='C[CH2]')

reactant1.thermo = ref_db.thermo.get_thermo_data(reactant1)
reactant2.thermo = ref_db.thermo.get_thermo_data(reactant2)
product1.thermo = ref_db.thermo.get_thermo_data(product1)
product2.thermo = ref_db.thermo.get_thermo_data(product2)

my_reaction = rmgpy.reaction.Reaction()
my_reaction.reactants = [reactant1, reactant2]
my_reaction.products = [product1, product2]

# # reverse it
# my_reaction.products = [reactant1, reactant2]
# my_reaction.reactants = [product1, product2]

In [139]:
def get_family(reaction):
    skip_list = [  # these aren't bimolecular reactions, so skip
        '1,2-Birad_to_alkene',
        '1,2_NH3_elimination',
        '1,2_XY_interchange',
        '1,2_shiftC',
        '1,2_shiftS',
        '1,4_Cyclic_birad_scission',
        '1,4_Linear_birad_scission',
        '6_membered_central_C-C_shift',
        'Baeyer-Villiger_step2',
        'Concerted_Intra_Diels_alder_monocyclic_1,2_shiftH',
        'Cyclic_Ether_Formation',
        'Cyclic_Thioether_Formation',
        'H2_Loss',
        'HO2_Elimination_from_PeroxyRadical',
        'Intra_2+2_cycloaddition_Cd',
        'Intra_5_membered_conjugated_C=C_C=C_addition',
        'Intra_Diels_alder_monocyclic',
        'Intra_Disproportionation',
        'Intra_RH_Add_Endocyclic',
        'Intra_RH_Add_Exocyclic',
        'Intra_R_Add_ExoTetCyclic',
        'Intra_R_Add_Exo_scission',
        'Intra_Retro_Diels_alder_bicyclic',
        'Intra_ene_reaction',
        'Korcek_step1',
        'Korcek_step2',
        'Singlet_Carbene_Intra_Disproportionation',
        'Singlet_Val6_to_triplet',
        'Surface_Bidentate_Dissociation',
        'Surface_Migration',
        'intra_H_migration',
        'intra_NO2_ONO_conversion',
        'intra_OH_migration',
        'intra_substitutionCS_cyclization',
        'intra_substitutionCS_isomerization',
        'intra_substitutionS_cyclization',
        'intra_substitutionS_isomerization',
    ]


    include_list = [  # these are bimoleculars
        'Disproportionation',
        'Disproportionation-Y',
        'F_Abstraction',
        'H_Abstraction', 
    ]
    for family in ref_db.kinetics.families:
#         if family in skip_list:
#             continue
        if family not in include_list:
            continue
            
            
#         print(f'Trying family: {family}')  # for debugging
        my_family = ref_db.kinetics.families[family]

        test_rxn = relabel(reaction, family)
        if test_rxn is False:
            continue

#         # print atom labels to check labeling
#         for sp in test_rxn.reactants + test_rxn.products:
#             print(sp.molecule[0].get_all_labeled_atoms())

        try:
            template = my_family.get_reaction_template(test_rxn)
            return family
        except rmgpy.exceptions.UndeterminableKineticsError:
            continue

In [138]:
# get_family(my_reaction)

get_family(reaction_list[1])

'H_Abstraction'